In [ ]:
import builtins
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
from deriva_ml import ExecutionConfiguration, MLVocab, DerivaSystemColumns, DatasetSpec, DerivaML, Workflow
from deriva_ml.demo_catalog import create_demo_catalog, DemoML
from IPython.display import display, Markdown, JSON
import itertools
import pandas as pd

In [ ]:
foo: int = 1
bar: str = "hello"

In [ ]:
print('foo', foo)
print('bar', bar)

In [ ]:
hostname = 'dev.eye-ai.org'
domain_schema = 'eye-ai'

gnl = GlobusNativeLogin(host=hostname)
if gnl.is_logged_in([hostname]):
    print("You are already logged in.")
else:
    gnl.login([hostname], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")


In [ ]:
ml_instance = DemoML(hostname, domain_schema)
print(f'Creating catalog at {ml_instance.catalog_id}')

ml_instance.add_term(MLVocab.workflow_type, "Manual Workflow", description="Initial setup of Model File")
ml_instance.add_term(MLVocab.asset_type, "API_Model", description="Model for our API workflow")

In [ ]:
api_workflow = ml_instance.create_workflow(
    name="Manual Workflow",
    workflow_type="Manual Workflow",
    description="A manual operation"
)

In [ ]:
manual_execution = ml_instance.create_execution(ExecutionConfiguration( description="Sample Execution", workflow=api_workflow))
manual_execution.upload_execution_outputs()
# Now lets create model configuration for our program.